In [11]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

variables_electrodos = {'FrontalAnteriorAsy': ["Fp2Fp1", "F4F3"],
    'FrontalAsy': ["Fp2Fp1", "F4F3", "F8F7"],
    'FrontalAsy1': ["Fp2Fp1", "F8F7"],
    'FrontalAsy2': ["F8F7", "F4F3"],
    'FrontalCenAsy1': ["FC2FC1", "FC6FC5"],
    'FrontalALLAsy2': ["FC2FC1", "FC6FC5", "Fp2Fp1", "F4F3", "F8F7"],
    'CenFAsy': ["C4C3", "FC6FC5", "FC2FC1"],
    'CenALLAsy': ["C4C3", "FC6FC5", "FC2FC1", "CP6CP5", "CP2CP1"],
    'CentPAsy': ["CP2CP1", "CP6CP5"],
    'CentPAsy2': ["CP2CP1", "CP6CP5"],
    'TemPAsy': ["T8T7", "TP10TP9", "P4P3", "P8P7"],
    'TemAsy': ["T8T7", "TP10TP9", "FT10FT9"],
    'ParietalAsy1': ["P4P3", "P8P7"],
    'ParietalAllAsy': ["P4P3", "TP10TP9", "P8P7", "CP2CP1", "CP6CP5"],
    'CenterParietalAsy': ["P4P3", "P8P7", "CP6CP5"],
    'MeanAsy': ["Fp2Fp1", "F4F3", "F8F7", "FT10FT9", "FC2FC1", "FC6FC5", "C4C3", "T8T7", "TP10TP9", "CP2CP1", "CP6CP5", "P4P3", "P8P7", "O2O1"],
}

# read data from excel file 
df = pd.read_excel('ET_ToDoModerations.xlsx', sheet_name='Tabelle1')

# convert columns to numeric
columns_to_convert = ['MeanEntO']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# choose target columns and set index to subject
df_filtered = df[['Subject','MeanEntO']]
df_filtered.set_index('Subject', inplace=True)
df_filtered.head()


,MeanEntO
Subject,
1,3.857143
3,4.000000
4,6.142857
5,4.000000
8,4.000000


In [3]:
# Load data from Excel files
data = pd.read_excel('CRB_Power_B1B2.xlsx')

# Define electrode pairs for calculation
electrode_pairs = [
    ('Fp2', 'Fp1'), ('F4', 'F3'), ('F8', 'F7'), ('FT10', 'FT9'), ('FC2', 'FC1'),
    ('FC6', 'FC5'), ('C4', 'C3'), ('T8', 'T7'), ('TP10', 'TP9'), ('CP2', 'CP1'),
    ('CP6', 'CP5'), ('P4', 'P3'), ('P8', 'P7'), ('O2', 'O1')
]

# Function to calculate the differences for each pair and condition
def calculate_differences(data, pairs, suffix):
    results = pd.DataFrame()
    for pair in pairs:
        diff_col_name = pair[0] + pair[1] + suffix  # e.g., Fp2Fp1_B1
        results[diff_col_name] = data[pair[0] + suffix] - data[pair[1] + suffix]
    return results

# Calculate differences for B1, B2, and B1B2 conditions
new_data = pd.DataFrame()
for suffix in ['_B1', '_B2', '_B1B2']:
    diffs = calculate_differences(data, electrode_pairs, suffix)
    new_data = pd.concat([new_data, diffs], axis=1)

# Add 'Subject', 'Pwelch', and 'Freq_range' columns from the original DataFrame
new_data['Subject'] = data['subid']
new_data['Pwelch'] = data['Freq_Names']
new_data['Freq_range'] = data['Freq_Range']

# Ensure the new DataFrame has the correct column order
cols = ['Subject', 'Pwelch', 'Freq_range'] + [col for col in new_data if col not in ['Subject', 'Pwelch', 'Freq_range']]
new_data_corrected = new_data[cols]

new_data_corrected.head(n=10)

,Subject,Pwelch,Freq_range,Fp2Fp1_B1,F4F3_B1,F8F7_B1,FT10FT9_B1,FC2FC1_B1,FC6FC5_B1,C4C3_B1,...,FC2FC1_B1B2,FC6FC5_B1B2,C4C3_B1B2,T8T7_B1B2,TP10TP9_B1B2,CP2CP1_B1B2,CP6CP5_B1B2,P4P3_B1B2,P8P7_B1B2,O2O1_B1B2
0,1,Alpha1,8-13,0.385810,0.443402,0.439939,-0.614480,0.124590,0.293298,0.383456,...,0.136076,0.220748,0.248142,0.175325,-0.050729,-0.051844,-0.057642,-0.042727,-0.228514,1.230284
1,1,Alpha2,7.5-12.5,0.432096,0.480357,0.377144,-0.425650,0.144446,0.332155,0.489876,...,0.156844,0.282109,0.333860,0.038117,0.049941,-0.060415,0.013004,0.033637,-0.130899,1.474093
2,1,Alpha3,7.5-10,0.625690,0.135172,-1.003521,-1.503887,0.075778,0.132816,0.498405,...,0.074695,0.059381,0.372077,-0.000401,-0.267669,0.032630,0.074106,0.194987,-0.126508,0.102870
3,1,Alpha4,10-12.5,0.035523,0.740321,0.280599,-0.449304,0.217385,0.571834,0.768361,...,0.243683,0.538678,0.567926,-0.070628,0.371112,-0.108718,0.027038,-0.022079,0.304905,2.474690
4,1,Alpha5,7-13,-0.034177,0.118819,-1.107239,-0.628396,0.069245,0.151472,0.325979,...,0.075257,0.072267,0.200636,0.214444,0.063058,-0.059101,-0.002311,-0.047511,-0.284040,0.983036
5,1,Delta,1-4,34.268234,0.209883,-19.557881,-1.204527,0.476025,1.710448,1.471790,...,0.609187,-12.189248,1.591227,-41.253870,-38.478306,0.656645,2.775599,1.177840,-56106.653795,107.253919
6,1,Theta,4-8,5.225157,-0.240628,-1.537624,-1.527295,0.032393,-0.140996,0.108617,...,0.057612,-0.270726,0.116463,-0.374976,0.266953,-0.047893,0.222066,0.098869,-6.296020,5.100578
7,1,lowAlpha,7-10,-0.166730,-0.549286,-3.115462,-0.497411,-0.064249,-0.251404,0.041803,...,-0.078829,-0.341368,-0.039628,0.283878,-0.039284,-0.026188,0.104528,0.052737,-0.728801,-0.184686
8,1,highAlpha,10-12,0.141706,0.998132,2.448142,1.191705,0.247448,0.631163,0.477082,...,0.280067,0.616200,0.276536,0.095895,0.526355,-0.199981,-0.078649,-0.208387,-0.137485,3.530928
9,1,Beta,13-30,0.401737,-0.037405,0.300328,-0.282493,0.002414,0.157095,-0.009188,...,0.014156,0.158832,-0.009410,-0.147100,0.109768,-0.022162,-0.059261,-0.044406,-0.280174,0.007977


In [4]:
# Function to filter, merge, and re-index DataFrames based on condition
def process_condition(data, electrode, df_filtered, pwelch):
    # Filter data for the specified 'pwelch_condition'
    pwelch_data = [data[data['Pwelch'] == band][['Subject', electrode]] for band in pwelch]
    
    # Create a list of DataFrames for each electrode in the condition
    df_pwelch = [df_filtered.merge(band, left_on='Subject', right_on='Subject') for band in pwelch_data]
    
    # Set the index to 'Subject' for each DataFrame in the list
    df_pwelch_with_index = [df.set_index('Subject') for df in df_pwelch]
    
    return df_pwelch_with_index


In [14]:
sf = dict()

for region, electrodes in variables_electrodos.items():
   
    if len(electrodes) > 1:

        df_B1 = list()
        df_B2 = list()
        df_B1B2 = list()

        for electrode in electrodes:
            # df_B1.append(process_condition(new_data_corrected, electrode + '_B1', df_filtered, new_data_corrected.Pwelch.unique()))
            df_B2.append(process_condition(new_data_corrected, electrode + '_B2', df_filtered, new_data_corrected.Pwelch.unique()))
            # df_B1B2.append(process_condition(new_data_corrected, electrode + '_B1B2', df_filtered, new_data_corrected.Pwelch.unique()))
        
        df_bin = df_B2

        for i in range(1, len(df_bin)):
            for j in range(len(df_bin[0])):
            
                df2 = df_bin[i][j].drop(columns=['MeanEntO'])
                df_bin[0][j] = pd.concat([df_bin[0][j], df2], axis=1)

        df_combinado = df_bin[0]
        prob_f_statistic_list = list()

        for i in range(len(df_combinado) - 1): # todos menos Gamma
            
            columns_to_average = df_combinado[i].drop(columns=['MeanEntO'])
            df_combinado[i][f'{region}'] = columns_to_average.mean(axis=1)
            result_df = df_combinado[i][['MeanEntO', f'{region}']]

            # OLS Regression
            y = result_df['MeanEntO']
            X = result_df[f'{region}']
            X = sm.add_constant(X) # adding a constant
    
            # fit the model
            model = sm.OLS(y, X)
            results = model.fit()
            
            prob_f_statistic_list.append(results.f_pvalue)

        sf[region] = prob_f_statistic_list
        print(f'{region} - Probabilidad de F-Statistic: {prob_f_statistic_list}')

sf_df = pd.DataFrame(sf)
sf_df.to_excel('MeanEntO.xlsx', index=False)

FrontalAnteriorAsy - Probabilidad de F-Statistic: [0.8158194335041208, 0.8158194335041208, 0.8158194335041133, 0.8158193913347803, array(nan), array(nan), 0.8178985246561635, array(nan), 0.8158193947268662, 0.8157560550482872]
FrontalAsy - Probabilidad de F-Statistic: [0.8158194335011413, 0.8158194335011559, 0.8158194335019539, 0.815791573213552, array(nan), array(nan), 0.9275208332016232, array(nan), 0.8156024859052398, 0.8154762878754338]
FrontalAsy1 - Probabilidad de F-Statistic: [0.8158194335011413, 0.8158194335011559, 0.8158194335019539, 0.8157914816387549, array(nan), array(nan), 0.9275279593422289, array(nan), 0.8156019478211409, 0.8153491472550677]
FrontalAsy2 - Probabilidad de F-Statistic: [0.017865858106533756, 0.0171975542956244, 0.04085602194202667, 0.19834211310513378, 0.04260138035520924, array(nan), 0.9311820068525563, 0.08127574745329476, 0.22783638179612986, 0.8464156899610682]
FrontalCenAsy1 - Probabilidad de F-Statistic: [0.012330560846792809, 0.012523608443581042, 0